# Model Building
This notebook takes the features extracted in feature_extraction.ipynb and uses them to train and test various models with various parameters to establish the 'best-fit' model that can then be saved and used for the prediction pipeline

In [1]:
#imports
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import time
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import random
import pickle
%matplotlib inline

## model 101: build a 'bog-standard' linear svm

In [2]:
# let's load the pre-processed training data
filename = 'data/training_data'
temp = open(filename, 'r+b')
try:
    training_data = np.load(filename)
    training_data.files
finally:
    temp.close()

# some stats
X = training_data['X']
y = training_data['y']

print('we have {} feature vectors with {} features per image'.format(len(X), len(X[0])))

we have 17760 feature vectors with 8412 features per image


In [4]:
# build a simple model
def split_and_scale_data(X, y, test_size=0.2):
    rand_state = np.random.randint(0, 100)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=rand_state)

    # Fit a per-column scaler
    X_scaler = StandardScaler().fit(X_train)
    # Apply the scaler to X
    X_train = X_scaler.transform(X_train)
    X_test = X_scaler.transform(X_test)

    print('Feature vector length:', len(X_train[0]))

    return X_train, y_train, X_test, y_test, X_scaler

def build_model(clf, X_train, y_train):
    # Check the training time for the SVC
    t=time.time()
    clf.fit(X_train, y_train)
    t2 = time.time()
    print(round(t2-t, 2), 'Seconds to train...')
    
    return clf

def test_model(clf, X_test, y_test):
    # Check the score of the model
    t=time.time()
    print('Test Accuracy of model = ', round(clf.score(X_test, y_test), 4))
    # Check the prediction time for a single sample
    t2=time.time()
    print(round(t2-t, 2), 'Seconds to predict all test image vectors...')
    
X_train, y_train, X_test, y_test, X_scaler = split_and_scale_data(X, y)

# let's try bog standard svc, gaussian bayes, decision tree

# Use a linear SVC 
print('SVC...')
svc = svm.SVC(kernel='linear')
svc = build_model(clf, X_train, y_train)
test_model(svc, X_test, y_test)

# Gaussian Bayes
print('Gaussian Bayes...')
clf = GaussianNB()
clf = build_model(clf, X_train, y_train)
test_model(clf, X_test, y_test)

# decision tree
print('Decision Tree...')
clf = tree.DecisionTreeClassifier()
clf = build_model(clf, X_train, y_train)
test_model(clf, X_test, y_test)

Feature vector length: 8412
SVC...
338.36 Seconds to train...
Test Accuracy of model =  0.944
0.12 Seconds to predict all test image vectors...
Gaussian Bayes...
2.53 Seconds to train...
Test Accuracy of model =  0.8181
1.0 Seconds to predict all test image vectors...
Decision Tree...
350.35 Seconds to train...
Test Accuracy of model =  0.9389
0.12 Seconds to predict all test image vectors...


The important points here are that the feature vectors were shuffled into a training and test set at random (although the time series data in the training set calls for a more robust method of splitting). The training data was scaled using the standard scalar (zero mean and unit variance). The test data was then separately scaled using the fitted scalar.

In addition, it's clear that out of the box, the linear svm has a really good initial testing accuracy and is relatively quick to train and very quick to predict. The Gaussian Bayes accuracy is lower, whilst the decision tree has good accuracy but takes a long time to train cf. svm.

It's time to tune the models and create a pickle output for the best model so it can be loaded back in at a later date with the feature extraction parameters and scalar for prediction

## model tuning and pickling

In [5]:
def pickle_model(clf, scaler, params):
    # create a dictionary to pickle
    pickled_objects = {
        'model': clf,
        'scaler': scaler,
        'params': params
    }
    
    # create output file
    filename = 'model.pkl'
    output = open(filename, 'wb')

    # Pickle dictionary using protocol 0.
    pickle.dump(pickled_objects, output)

    output.close()
    
    return filename

In [7]:
# let's try a pickle of the bog standard svc

filename = pickle_model(svc, X_scaler, None)

pkl_file = open(filename, 'rb')

data1 = pickle.load(pkl_file)
print(data1)
pkl_file.close()

{'scaler': StandardScaler(copy=True, with_mean=True, with_std=True), 'params': None, 'model': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')}


tuning models involve cycling through combinations of hyper-paramters for each classifier

In [ ]:
# tuning the SVC - we can tune kernel and 'c' - will split into 2
parameters = {'kernel':('sigmoid', 'rbf'), 'C':[1, 10], 'gamma':[0.0001, 0.005]}
svc = svm.SVC()
clf = GridSearchCV(svc, parameters)
clf = build_model(clf, X_train, y_train)
print(sorted(clf.cv_results_.keys()))
test_model(clf, X_test, y_test)
print(clf.best_params_)

In [ ]:
parameters = {'kernel':('linear', 'poly'), 'C':[1, 10], 'gamma':[0.0001, 0.005]}
svc = svm.SVC()
clf = GridSearchCV(svc, parameters)
clf = build_model(clf, X_train, y_train)
print(sorted(clf.cv_results_.keys()))
test_model(clf, X_test, y_test)
print(clf.best_params_)

In [ ]:
# tuning the decision tree - we can tune criterion, max-depth and min samples split - will split into 3
parameters = {'criterion':('gini', 'entropy'), 'max_depth':[4], 'min_samples_split':[2, 5, 10]}
tree = tree.DecisionTreeClassifier()
clf = GridSearchCV(tree, parameters)
clf = build_model(clf, X_train, y_train)
print(sorted(clf.cv_results_.keys()))
test_model(clf, X_test, y_test)
print(clf.best_params_)

In [ ]:
# tuning the decision tree - we can tune criterion, max-depth and min samples split - will split into 3
parameters = {'criterion':('gini', 'entropy'), 'max_depth':[8], 'min_samples_split':[2, 5, 10]}
tree = tree.DecisionTreeClassifier()
clf = GridSearchCV(tree, parameters)
clf = build_model(clf, X_train, y_train)
print(sorted(clf.cv_results_.keys()))
test_model(clf, X_test, y_test)
print(clf.best_params_)

In [ ]:
# tuning the decision tree - we can tune criterion, max-depth and min samples split - will split into 3
parameters = {'criterion':('gini', 'entropy'), 'max_depth':[10], 'min_samples_split':[2, 5, 10]}
tree = tree.DecisionTreeClassifier()
clf = GridSearchCV(tree, parameters)
clf = build_model(clf, X_train, y_train)
print(sorted(clf.cv_results_.keys()))
test_model(clf, X_test, y_test)
print(clf.best_params_)

This gives us...